# MNIST Digit Classification with KNN and Naive Bayes

In [0]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

In [0]:
import sklearn
sklearn.__version__

Load the data. Notice that the data gets partitioned into training, development, and test sets. Also, a small subset of the training data called mini_train_data and mini_train_labels gets defined, which you should use in all the experiments below, unless otherwise noted.

In [0]:
# Load the digit data from https://www.openml.org/d/554 or from default local location '~/scikit_learn_data/...'
X, Y = fetch_openml(name='mnist_784', return_X_y=True, cache=False)


# Rescale grayscale values to [0,1].
X = X / 255.0

# Shuffle the input: create a random permutation of the integers between 0 and the number of data points and apply this
# permutation to X and Y.
# NOTE: Each time you run this cell, you'll re-shuffle the data, resulting in a different ordering.
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

print('data shape: ', X.shape)
print('label shape:', Y.shape)

# Set some variables to hold test, dev, and training data.
test_data, test_labels = X[61000:], Y[61000:]
dev_data, dev_labels = X[60000:61000], Y[60000:61000]
train_data, train_labels = X[:60000], Y[:60000]
mini_train_data, mini_train_labels = X[:1000], Y[:1000]

### Part 1:

Show a 10x10 grid that visualizes 10 examples of each digit.


In [0]:
def P1(num_examples):

  fig = plt.figure(figsize=(num_examples*3,num_examples*3))

  for x in range(num_examples):
    for y in range(num_examples):
      ax = fig.add_subplot(num_examples, num_examples, num_examples*y+x+1)
      plt.rc('image', cmap='gray')
      ax.matshow(X[np.where(Y == str(y))[0][x]].reshape((28,28)))
      plt.title('Digit Label: {}'.format(str(y)))
      plt.xticks(np.array([]))
      plt.yticks(np.array([]))

P1(10)

### Part 2:

Produce k-Nearest Neighbors models with k $\in$ [1,3,5,7,9].  Evaluate and show the accuracy of each model.


In [0]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

def P2(k_values):

  for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(mini_train_data, mini_train_labels)
    y_pred = knn.predict(dev_data)

    acc = accuracy_score(dev_labels, y_pred)
    print('k = %d, accuracy: %3.3f' %(k, acc))

    if k == 1:
      print(classification_report(dev_labels, y_pred))

k_values = [1, 3, 5, 7, 9]
P2(k_values)

### Part 3:

Produce 1-Nearest Neighbor models using training data of various sizes.  Evaluate and show the performance of each model.  Additionally, show the time needed to measure the performance of each model.


In [0]:
import timeit
from sklearn.metrics import accuracy_score

def P3(train_sizes, accuracies):

  for sizes in train_sizes:
    
    start = timeit.default_timer()
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(train_data[0:sizes], train_labels[0:sizes])
    y_pred = knn.predict(dev_data)
    stop = timeit.default_timer()
    acc = accuracy_score(dev_labels, y_pred)
    accuracies.append(acc)
    print('train sizes = %d, accuracy: %3.3f, time: %s' %(sizes, acc, stop - start))

train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600]
accuracies = []
P3(train_sizes, accuracies)

### Part 4:

Produce a linear regression model that predicts accuracy of a 1-Nearest Neighbor model given training set size. Show $R^2$ of the linear regression model.  Show the accuracies predicted for training set sizes 60000, 120000, and 1000000.  Show a lineplot of actual accuracies and predicted accuracies vs. training set size over the range of training set sizes in the training data.



In [0]:
def P4():

  train_sizes = [100, 200, 400, 800, 1600, 3200, 6400, 12800, 25600, 60000, 120000, 1000000]
  accuracies = []

  for sizes in train_sizes:
    
    knn = KNeighborsClassifier(n_neighbors=1)
    knn.fit(train_data[0:sizes], train_labels[0:sizes])
    y_pred = knn.predict(dev_data)
    acc = accuracy_score(dev_labels, y_pred)
    accuracies.append(acc)
    print('train sizes = %d, accuracy: %3.3f' %(sizes, acc))

  sizes = [[100], [200], [400], [800], [1600], [3200], [6400], [12800], [25600], [60000],[120000],[1000000]]
  sizes = np.array(sizes)  

  # Untransform
  lr = LinearRegression()
  lr.fit(sizes, accuracies)
  print("Untransform model, R-squared: ",lr.score(sizes, accuracies))

  # Initialize a new plot.
  plt.figure(figsize=(8, 4))

  # Show samples from the fitted function.
  plt.plot(sizes, lr.predict(sizes), label="Untransform model")

  # Show the original noisy samples.
  plt.scatter(sizes, accuracies, label="Accuracies")

  # Add a few more labels to the plot.
  plt.xlabel("Training set sizes")
  plt.ylabel("Accuracy")
  plt.legend(loc="best")

  # Render the plots.
  plt.show()


  # Transform
  lr = LinearRegression()
  lr.fit(np.log(sizes), accuracies)
  print("Log transform model, R-squared: ",lr.score(np.log(sizes), accuracies))

  # Initialize a new plot.
  plt.figure(figsize=(8, 4))

  # Show samples from the fitted function.
  plt.plot(np.log(sizes), lr.predict(np.log(sizes)), label="Log transform model")

  # Show the original noisy samples.
  plt.scatter(np.log(sizes), accuracies, label="Accuracies")

  # Add a few more labels to the plot.
  plt.xlabel("Training set sizes (2^x)")
  plt.ylabel("Accuracy")
  plt.legend(loc="best")

  # Render the plots.
  plt.show()


P4()

### Part 5:

Produce a 1-Nearest Neighbor model and show the confusion matrix.  Show the images of these most often confused digits.


In [0]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

def P5():

  k = 1
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(mini_train_data, mini_train_labels)
  y_pred = knn.predict(dev_data)

  print("The confusion matrix of 1-Nearest Neighbor model.")
  print(confusion_matrix(dev_labels, y_pred))
  print()
  result = confusion_matrix(dev_labels, y_pred)

  error_num = []
  for i in range(len(result)):
    error_num.append(result[i].sum() - result[i][i])

  digits = [i for i, x in enumerate(error_num) if x == max(error_num)]

  for i in digits:
    print(i, "is the digit that 1-Nearest Neighbor confuse the most.")

  
  # error digits in 2 & 8
  index2 = list(np.where((y_pred!= dev_labels) & (dev_labels == "2"))[0])
  index8 = list(np.where((y_pred!= dev_labels) & (dev_labels == "8"))[0])
  index = index2 + index8

  for i in range(len(index)):
    plt.rc('image', cmap='gray')
    img = dev_data[index[i]].reshape((28,28))
    plt.title('Label: {}, Predict: {}'.format(str(dev_labels[index[i]]), str(y_pred[index[i]])))
    plt.imshow(img)
    plt.show()

P5()

### Part 6:
Smooth an image by blurring.

In [0]:
from scipy.ndimage.filters import gaussian_filter

def P6():
    
### STUDENT START ###
  k = 1
  blur_mini_train_data = gaussian_filter(mini_train_data, sigma = 1, order = 0, mode = "constant", truncate = 1, cval = 2.0)
  blur_dev_data = gaussian_filter(dev_data, sigma = 1)

  # no filter
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(mini_train_data, mini_train_labels)
  y_pred = knn.predict(dev_data)
  print("No filter, accuracy: ", accuracy_score(dev_labels, y_pred))

  # Filter the training data but not the dev data
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(blur_mini_train_data, mini_train_labels)
  y_pred = knn.predict(dev_data)
  print("Filter the training data but not the dev data, accuracy: ", accuracy_score(dev_labels, y_pred))

  # Filter the dev data but not the training data
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(mini_train_data, mini_train_labels)
  y_pred = knn.predict(blur_dev_data)
  print("Filter the dev data but not the training data, accuracy: ", accuracy_score(dev_labels, y_pred))

  # Filter both training data and dev data
  knn = KNeighborsClassifier(n_neighbors=k)
  knn.fit(blur_mini_train_data, mini_train_labels)
  y_pred = knn.predict(blur_dev_data)
  print("Filter both training data and dev data, accuracy: ", accuracy_score(dev_labels, y_pred))

### STUDENT END ###

P6()

### Part 7:

Produce two Naive Bayes models and evaluate their performances.


In [0]:
def P7():

  # Bernoulli model
  #prepocess mini_train_data
  b_mini_train = mini_train_data[:]
  b_mini_train[b_mini_train >= 0.1] = 1
  b_mini_train[b_mini_train < 0.1] = 0

  #prepocess dev_data
  b_dev = dev_data[:]
  b_dev[b_dev >= 0.1] = 1
  b_dev[b_dev < 0.1] = 0

  # fit model
  clf = BernoulliNB()
  clf.fit(b_mini_train, mini_train_labels)
  print ('Bernoulli model accuracy: %3.3f' %clf.score(b_dev, dev_labels))

  # Multinomial model
  #prepocess mini_train_data
  m_mini_train = mini_train_data[:]
  m_mini_train[m_mini_train >= 0.9] = 2
  m_mini_train[(m_mini_train >= 0.1) & (m_mini_train < 0.9)] = 1
  m_mini_train[m_mini_train < 0.1] = 0

  #prepocess dev_data
  m_dev = dev_data[:]
  m_dev[m_dev >= 0.9] = 2
  m_dev[(m_dev >= 0.1) & (m_dev < 0.9)] = 1
  m_dev[m_dev < 0.1] = 0

  # fit model

  clf = MultinomialNB()
  clf.fit(m_mini_train, mini_train_labels)
  print ('Multinomial model accuracy: %3.3f' %clf.score(m_dev, dev_labels))

P7()

### Part 8:

Search across several values of the LaPlace smoothing parameter (alpha) to find its effect on a Bernoulli Naive Bayes model's performance.  Show the accuracy at each alpha value.

In [0]:
def P8(alphas):

  b_nb = BernoulliNB(binarize=0.0)
  clf = GridSearchCV(b_nb, alphas, cv= 5, scoring='accuracy', iid=False)
  clf.fit(mini_train_data, mini_train_labels)

  
  print ('accuracy: %3.3f' %clf.score(dev_data, dev_labels))
  clf.best_params_

  return clf

alphas = {'alpha': [1.0e-10, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 2.0, 10.0]}
nb = P8(alphas)
print()
print("Best alpha = ", nb.best_params_)

### Part 9:

Produce a model using Guassian Naive Bayes, which is intended for real-valued features, and evaluate performance.


In [0]:
def P9():

  gnb = GaussianNB()
  gnb_fit = gnb.fit(mini_train_data, mini_train_labels)
  print('Accuracy: %f' %gnb.score(dev_data, dev_labels))

P9()

### Part 10:

Naive Bayes produces a generative model, use it to generate digit images.


In [0]:
def P10(num_examples):

  b_nb = BernoulliNB(binarize=0.5)
  b_nb_fit = b_nb.fit(mini_train_data, mini_train_labels)

  def trans_image(pix_prob):
      return(np.where(np.random.rand(1, 784) < np.exp(pix_prob),1, 0))

  # Generate plots
  for index, number in enumerate(b_nb_fit.feature_log_prob_, 0):
      for rep in range(0, num_examples):
          plot_number = (rep + (index * num_examples)) + 1
          plt.subplot(10,num_examples,plot_number)
          frame1 = plt.gca()
          frame1.axes.get_xaxis().set_visible(False)
          frame1.axes.get_yaxis().set_visible(False)
          plt.imshow(trans_image(number).reshape(28,28), cmap = 'Greys')

P10(20)

### Part 11:

Recall that a strongly calibrated classifier is rougly 90% accurate when the posterior probability of the predicted class is 0.9. A weakly calibrated classifier is more accurate when the posterior probability of the predicted class is 90% than when it is 80%. A poorly calibrated classifier has no positive correlation between posterior probability and accuracy.  


In [0]:
def P11(buckets, correct, total):
    
  index = 0

  for i in buckets:
    b_nb = BernoulliNB(alpha = 0.01, binarize=0.0)
    clf = b_nb.fit(mini_train_data, mini_train_labels)
    preds = b_nb.predict(dev_data[0:int(dev_data.shape[0]*i)]) 
    correct[index] = np.sum(preds == dev_labels[0:int(dev_data.shape[0]*i)])
    total[index] = int(dev_data.shape[0]*i)
    index += 1

buckets = [0.5, 0.9, 0.999, 0.99999, 0.9999999, 0.999999999, 0.99999999999, 0.9999999999999, 1.0]
correct = [0 for i in buckets]
total = [0 for i in buckets]

P11(buckets, correct, total)

for i in range(len(buckets)):
  accuracy = 0.0
  if (total[i] > 0): accuracy = correct[i] / total[i]
  print('p(pred) is %.13f to %.13f    total = %3d    accuracy = %.3f' % (0 if i==0 else buckets[i-1], buckets[i], total[i], accuracy))